In [129]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import datetime
import os
from pandas.io.json import json_normalize

import warnings
warnings.filterwarnings('ignore')

In [130]:
startDate = "2017-01-01"
endDate = "2017-02-01"
resultsPerRequest = 100000
saveToDirectory = "data"

In [131]:
def buildURL(startDate, endDate, resultsPerRequest):
    startDate = startDate + "T00:00:00.000"
    endDate = endDate + "T00:00:00.000"
    url = "https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >='{}' and trip_start_timestamp < '{}'&$limit={}".format(startDate, endDate, resultsPerRequest)
    return(url)   

In [132]:
def callAPI(url):
    headers = {"X-App-Token": "bjp8KrRvAPtuf809u1UXnI0Z8"}
    res = requests.get(url, headers = headers, verify = False)
    print("API Response code: ", res.status_code)
    if (res.status_code == 200):
        results = res.json()
        return(results)
    else:
        return(None)

In [133]:
def gatherData(startDate, endDate, resultsPerRequest, saveToDirectory):
    offset = 0
    previousRecordsFound = 999
    
    while(previousRecordsFound > 0):
        url = buildURL(startDate, endDate, resultsPerRequest)
        url = url + "&$offset={}".format(offset)
        
        print("Calling offset {}: {}".format(offset, url))
        results = callAPI(url)
        
        if (results == None):
            previousRecordsFound = 0
        else:            
            previousRecordsFound = len(results)
            print("Results Found:", previousRecordsFound)

            if (previousRecordsFound > 0):
                df = json_normalize(results)
                fileName = "ChiTaxi_" + startDate + "_" + endDate + "_" + str(offset).zfill(20) + ".csv"
                print("Saving {} records to fileName: {}".format(previousRecordsFound, fileName))
                df.to_csv(os.path.join(saveToDirectory, fileName), index = False)
                offset = offset + 1
                time.sleep(30)
                print("--------------------------------------------------------------------------------------------------------")        

In [134]:
gatherData(startDate, endDate, resultsPerRequest, saveToDirectory)

Calling offset 0: https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >='2017-01-01T00:00:00.000' and trip_start_timestamp < '2017-02-01T00:00:00.000'&$limit=100000&$offset=0
API Response code:  200
Results Found: 100000
Saving 100000 records to fileName: ChiTaxi_2017-01-01_2017-02-0100000000000000000000.csv
--------------------------------------------------------------------------------------------------------
Calling offset 1: https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >='2017-01-01T00:00:00.000' and trip_start_timestamp < '2017-02-01T00:00:00.000'&$limit=100000&$offset=1


KeyboardInterrupt: 